In [1]:
# import modules
import os
from onto_vae.ontobj import *
from onto_vae.vae_model import *

In [2]:
# initialize the Ontobj
# the description should be an identifier, e.g. the ontology used, here: PWO (Pathway Ontology)
pwo = Ontobj(description='PWO')

In [3]:
1000/128

7.8125

In [4]:
# initialize our ontology
# obo: path to an obo file
# gene_annot: path to a tab separated file with two columns: Genes and Ontology IDs
pwo.initialize_dag(obo=data_path() + 'pw.obo',
                   gene_annot=data_path() + 'gene_term_mapping.txt')

# trim the ontology
pwo.trim_dag(top_thresh=1000, 
             bottom_thresh=30)

#create masks for decoder initialization

################### parameter which positions to change ##########################

In [5]:
expr_data = data_path() + 'pbmc_sample_expr.csv'
pd.read_csv(expr_data)

,gene_symbol,AAACATACCCGTAA-1-stimulated,AAACATACGAGGTG-1-stimulated,AAACATACGTCGTA-1-stimulated,AAACATTGACTTTC-1-stimulated,AAACATTGTGCTAG-1-stimulated,AAACATTGTGTGGT-1-stimulated,AAACCGTGCTGGAT-1-stimulated,AAACGCTGAGCACT-1-stimulated,AAACGCTGGCCCTT-1-stimulated,...,AGTCAGACTGCATG-1-control,AGTCGAACGGTTAC-1-control,AGTCGCCTTCGACA-1-control,AGTCGCCTTGGCAT-1-control,AGTCTACTATGCTG-1-control,AGTCTACTCGTGAT-1-control,AGTCTTACCTCAGA-1-control,AGTCTTACTTCGCC-1-control,AGTGAAGAACTGTG-1-control,AGTGACACCCTTAT-1-control
0,AL627309.1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,RP11-206L10.9,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,LINC00115,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,NOC2L,0.0,0.0,0.0,0.0,0.0,0.0,0.92,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.95
4,KLHL17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6993,SLC19A1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
6994,COL6A2,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
6995,FTCD,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
6996,DIP2A,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [6]:
reg = np.zeros((5892,657))
reg[0,0] = 1

pwo.create_masks(top_thresh=1000,
                 bottom_thresh=30,
                 reg_mask= reg)

pwo.match_dataset(expr_data = data_path() + 'pbmc_sample_expr.csv',
                  name='PBMC_CD4T')

In [7]:
[i.shape for i in pwo.masks["1000_30"]]

[(3, 76),
 (12, 79),
 (131, 91),
 (169, 222),
 (160, 391),
 (101, 551),
 (5, 652),
 (5892, 657)]

In [8]:
# initialize OntoVAE 
pwo_model = OntoVAE(ontobj=pwo,              # the Ontobj we will use
                    dataset='PBMC_CD4T',     # which dataset from the Ontobj to use for model training
                    top_thresh=1000,         # which trimmed version to use
                    bottom_thresh=30)        # which trimmed version to use     
pwo_model.to(pwo_model.device)         

OntoVAE(
  (encoder): Encoder(
    (encoder): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=5892, out_features=228, bias=True)
        (1): BatchNorm1d(228, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Dropout(p=0.2, inplace=False)
        (3): ReLU()
      )
    )
    (mu): Sequential(
      (0): Linear(in_features=228, out_features=228, bias=True)
      (1): Dropout(p=0.5, inplace=False)
    )
    (logvar): Sequential(
      (0): Linear(in_features=228, out_features=228, bias=True)
      (1): Dropout(p=0.5, inplace=False)
    )
  )
  (decoder): OntoDecoder(
    (decoder): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=228, out_features=9, bias=True)
      )
      (1): Sequential(
        (0): Linear(in_features=237, out_features=36, bias=True)
      )
      (2): Sequential(
        (0): Linear(in_features=273, out_features=393, bias=True)
      )
      (3): Sequential(
        (0): Linear(in_features=666, out_fe

In [9]:
[i.shape for i in pwo.masks["1000_30"]]

[(3, 76),
 (12, 79),
 (131, 91),
 (169, 222),
 (160, 391),
 (101, 551),
 (5, 652),
 (5892, 657)]

In [18]:
# train the model
pwo_model.train_model(os.getcwd() + 'models/best_model.pt',   # where to store the best model
                     lr=1e-4,                                 # the learning rate
                     l1 = 1,                                  # the lambda coefficient
                     kl_coeff=1e-4,                           # the weighting coefficient for the Kullback Leibler loss
                     batch_size=128,                          # the size of the minibatches
                     epochs=5,                                # over how many epochs to train
                     log=False)    

Epoch 1 of 5


  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 121.97it/s]


New best model!
Train Loss: 13956.5214
Val Loss: 12119.7705
Epoch 2 of 5


100%|██████████| 2/2 [00:00<00:00, 122.74it/s]


...

Train Loss: 14036.7944
Val Loss: 11962.8672
Epoch 4 of 5


100%|██████████| 2/2 [00:00<00:00, 111.70it/s]


Train Loss: 13874.0869
Val Loss: 12053.5610
Epoch 5 of 5


100%|██████████| 2/2 [00:00<00:00, 105.90it/s]

Train Loss: 13823.3909
Val Loss: 12061.1567


In [11]:
pwo_model.decoder.decoder[-1][0].weight.data

tensor([[0.0035, 0.0000, 0.0129,  ..., -0.0000, -0.0000, 0.0000],
        [0.0000, 0.0000, -0.0000,  ..., 0.0000, -0.0000, -0.0000],
        [0.0000, 0.0000, -0.0000,  ..., 0.0000, 0.0000, -0.0000],
        ...,
        [0.0000, 0.0000, -0.0000,  ..., 0.0000, 0.0000, -0.0000],
        [-0.0000, 0.0000, -0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [-0.0000, -0.0000, 0.0000,  ..., 0.0000, 0.0000, -0.0000]])